# Sample satellite data

Some graphical tests of current primary functions using the sample satellite data.

In [ ]:
import sys; sys.path.append("../")
import warnings

import matplotlib.pyplot as plt
import numpy as np
import xarray as xr

import tams

In [ ]:
plt.close("all")

xr.set_options(display_expand_data=False)

## Load data

In [ ]:
r = tams.load_example_ir()

tb = tams.tb_from_ir(r, ch=9).isel(time=slice(4))
tb

## Identify cloud elements (CEs)

In [ ]:
times = tb.time
contour_sets = [tams.identify(tb.sel(time=time)) for time in times]
contour_sets[0].head()

## Track CE groups between times

In [ ]:
cs = tams.track(contour_sets, times, u_projection=-5)

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

ax.set(xlabel="lon", ylabel="lat")

colors = plt.cm.GnBu(np.linspace(0.2, 0.85, len(times)))

# Plot blobs at each time
for i, g in cs.groupby("itime"):
    color = colors[i]

    g.plot(ax=ax, facecolor=color, edgecolor=color, alpha=0.25, lw=1.5)

    # Label blobs with assigned ID
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore",
            category=UserWarning,
            message="Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect.",
        )
        for id_, x, y in zip(g.id, g.centroid.x, g.centroid.y):
            ax.text(x, y, id_, c=color, fontsize=14, zorder=10)


fig.tight_layout()

## Classify

In [ ]:
classes = cs.groupby("id").apply(tams.classify)
cs["class"] = cs.id.map(classes)
cs